## Phishing_Email_classification

Le code fourni est un script Python qui utilise des techniques d'apprentissage automatique pour classer les e-mails en tant que "Phishing Email" ou "Safe Email" (e-mail sécurisé). Ce script utilise un modèle XGBoost avec des vecteurs TF-IDF pour classifier les e-mails en tant que phishing ou sécurisés. La validation croisée est utilisée pour évaluer les performances du modèle, et une prédiction d'exemple est fournie à la fin du script.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier

# Cette section comprend les imports nécessaires pour le projet, notamment les bibliothèques Pandas, Matplotlib et les modules de Scikit-learn pour l'apprentissage automatique.

def load_data(file_path):
    data = pd.read_csv(file_path, encoding='latin-1')
    data.dropna(inplace=True)

    # Convert 'Phishing Email' to 1 and 'Safe Email' to 0
    data['Email Type'] = data['Email Type'].map({'Phishing Email': 1, 'Safe Email': 0})

    return data['Email Text'].values, data['Email Type'].values

# La fonction load_data prend le chemin du fichier CSV en entrée, charge les données à partir du fichier, et effectue une conversion des étiquettes en 1 pour 'Phishing Email' et 0 pour 'Safe Email'.
# Elle renvoie les textes d'e-mails (Email Text) et les étiquettes correspondantes (Email Type).

def train_model(X_train, y_train, X_val, y_val):
    vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1, 3), max_features=10000)
    vectorizer.fit(X_train)

    X_train_transformed = vectorizer.transform(X_train)
    X_val_transformed = vectorizer.transform(X_val)

    model = XGBClassifier(n_estimators=800, learning_rate=0.1, max_depth=4, colsample_bytree=0.2, n_jobs=-1, random_state=42)
    model.fit(X_train_transformed, y_train)

    y_pred = model.predict(X_val_transformed)

    return y_pred

# La fonction train_model prend les ensembles d'entraînement (X_train, y_train) et de validation (X_val, y_val) en entrée.
# Elle utilise TfidfVectorizer pour transformer les textes d'e-mails en vecteurs TF-IDF, puis entraîne un modèle XGBoost avec les paramètres spécifiés.
# La fonction renvoie les prédictions sur l'ensemble de validation.


def print_evaluation(y_true, y_pred, fold):
    print(f'Fold: {fold}')
    print(f'Accuracy Score: {accuracy_score(y_true, y_pred)}')
    print(f'Confusion Matrix: \n {confusion_matrix(y_true, y_pred)}')
    print(f'Classification Report: \n {classification_report(y_true, y_pred)}')


# La fonction print_evaluation affiche les métriques d'évaluation telles que la précision, la matrice de confusion et le rapport de classification pour chaque fold de la validation croisée.


def main():
    file_path = '/content/Phishing_Email.csv'
    X, y = load_data(file_path)

    num_folds = 5
    kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

    fold = 1
    for train_index, val_index in kfold.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        y_pred = train_model(X_train, y_train, X_val, y_val)
        print_evaluation(y_val, y_pred, fold)
        fold += 1

    # Example prediction
    new_text = ["Hello, this is definitely not a scam"]
    new_text_transformed = vectorizer.transform(new_text)
    predictions = model.predict(new_text_transformed)
    print(predictions)

if __name__ == "__main__":
    main()

# La fonction main est la partie principale du script. Elle charge les données, effectue une validation croisée stratifiée, entraîne le modèle, évalue les performances pour chaque fold, et effectue une prédiction d'exemple avec le modèle entraîné.